In [1]:
!pip install keras-tuner

In [20]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [66]:
x_train = pd.read_csv("/content/drive/MyDrive/DataForCheckpoint2/DataForCheckpoint2/Voltage_L1_train.csv")
y_train = pd.read_csv("/content/drive/MyDrive/DataForCheckpoint2/DataForCheckpoint2/output_train.csv")
x_test = pd.read_csv("/content/drive/MyDrive/DataForCheckpoint2/DataForCheckpoint2/Voltage_L1_test.csv")
y_test = pd.read_csv("/content/drive/MyDrive/DataForCheckpoint2/DataForCheckpoint2/output_test.csv")

In [67]:
print("x_train",x_train.shape)
print("y_train",y_train.shape)
print("x_test",x_test.shape)
print("y_test",y_test.shape)

x_train (5999, 256)
y_train (5999, 1)
x_test (3599, 256)
y_test (3599, 1)


In [68]:
header =[]
for i in range(1,x_train.shape[1]+1):
    header.append("Col"+str(i))
  
x_train.columns = header
x_test.columns = header

In [69]:
header = ["output"]
y_train.columns = header
y_test.columns = header

In [25]:
x_train.shape

(5999, 256)

In [26]:
x_train.shape

(5999, 256)

In [70]:
x_train = np.asarray(x_train).reshape(x_train.shape[0],x_train.shape[1], 1)
x_test = np.asarray(x_test).reshape(x_test.shape[0],x_test.shape[1], 1)

In [72]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv1D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(256,1)
    ),
    keras.layers.Conv1D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(7, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [73]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [75]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='nikhil',project_name="nik")

In [76]:
tuner_search.search(x_train,y_train,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 19s]
val_accuracy: 0.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 02m 33s
INFO:tensorflow:Oracle triggered exit


In [77]:
model=tuner_search.get_best_models(num_models=1)[0]

In [78]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 254, 48)           192       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 252, 112)          16240     
_________________________________________________________________
flatten (Flatten)            (None, 28224)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1354800   
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 343       
Total params: 1,371,575
Trainable params: 1,371,575
Non-trainable params: 0
_________________________________________________________________


In [79]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
169/169 [==============================] - 8s 43ms/step - loss: 0.2097 - accuracy: 0.9394 - val_loss: 1.1085e-06 - val_accuracy: 1.0000
Epoch 5/10
169/169 [==============================] - 7s 42ms/step - loss: 0.0518 - accuracy: 0.9848 - val_loss: 0.0113 - val_accuracy: 0.9950
Epoch 6/10
169/169 [==============================] - 7s 42ms/step - loss: 0.0678 - accuracy: 0.9815 - val_loss: 9.1460e-04 - val_accuracy: 1.0000
Epoch 7/10
169/169 [==============================] - 7s 42ms/step - loss: 0.0465 - accuracy: 0.9899 - val_loss: 7.0308e-04 - val_accuracy: 1.0000
Epoch 8/10
169/169 [==============================] - 7s 42ms/step - loss: 0.0283 - accuracy: 0.9933 - val_loss: 0.0041 - val_accuracy: 0.9983
Epoch 9/10
169/169 [==============================] - 7s 42ms/step - loss: 0.0152 - accuracy: 0.9969 - val_loss: 2.9561e-04 - val_accuracy: 1.0000
Epoch 10/10
169/169 [==============================] - 7s 42ms/step - loss: 0.0255 - accuracy: 0.9941 - val_loss: 0.0000e+00 -

In [80]:
pred_acc = model.evaluate(x_test,y_test)
print("Test accuracy is {}".format(pred_acc))

113/113 [==============================] - 1s 11ms/step - loss: 0.1849 - accuracy: 0.9675
Test accuracy is [0.18490825593471527, 0.9674909710884094]
